In [209]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import os.path as path
import random
from IPython.display import HTML
from collections import Counter, defaultdict
from itertools import combinations

import exdir
import numpy as np
import pandas as pd
from tqdm import tqdm
import skvideo
import yaml

os.chdir('/home/Gangus/cow-tus')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [235]:
data_dir = 'data/single-instance-learning/temporal-downsample'

hypothesis_conditions = ['by-animal-number', 'hold-out-validation']
group_dir = path.join('data', 'split', *hypothesis_conditions)

split_counts = {
    'train': 0, # will be replaced in the next line
    'valid': 15,
    'test': 15
}
split_counts['train'] = len(pd.read_csv(path.join(data_dir, 'attrs.csv'), index_col=0).index.unique()) - sum(split_counts.values())

strata_key = 'raw.animal_number'

In [237]:
attrs_path = path.join(data_dir, 'attrs.csv')
attrs_df = pd.read_csv(attrs_path, index_col=0)

labels = []
exam_groups = attrs_df.groupby('exdir.exam_id')
for exam_id, exam_group in exam_groups:
    labels.append(list(set(exam_group['preprocessed.global_label_multiclass']))[0])
label_freqs = sorted(Counter(labels).items())

split_to_quota = defaultdict(dict)
for label, freq in label_freqs:
    for split, count in split_counts.items():
        split_to_quota[split][label] = int((count / len(exam_groups)) * freq)

metadata_path = path.join(group_dir, 'metadata.yaml')
with open(metadata_path, 'w') as f:
    f.write(yaml.dump({'meta.estimated_counts': dict(split_to_quota)}))

animal_groups = attrs_df.groupby(strata_key)

animal_id_to_exam_ids = defaultdict(list)
for animal_id, animal_group in animal_groups:
    exam_ids = set(animal_group.index)
    animal_id_to_exam_ids[animal_id] += list(exam_ids)
    
animal_id_to_exam_ids = list(animal_id_to_exam_ids.items())
random.shuffle(animal_id_to_exam_ids)

In [241]:
split_to_exam_ids = defaultdict(list)
i = 0
for split, count in split_counts.items():
    added = 0
    while added < count:
        animal_id, exam_ids = animal_id_to_exam_ids[i]
        split_to_exam_ids[split] += exam_ids
        added += len(exam_ids)
        i += 1
    if count == -1:
        exam_ids_remaining = list(zip(*animal_id_to_exam_ids[i:]))[1]
        split_to_exam_ids[split] = [exam_id for exam_ids in exam_ids_remaining for exam_id in exam_ids]

In [242]:
split_to_labels = defaultdict(list)
for split, exam_ids in split_to_exam_ids.items():
    exam_groups = attrs_df.loc[exam_ids].groupby('exdir.exam_id')
    for exam_id, exam_group in exam_groups:
        split_to_labels[split].append(list(set(exam_group['preprocessed.global_label_multiclass']))[0])

In [243]:
for split, labels in split_to_labels.items():
    print(split, sorted(Counter(labels).items()))

train [(0, 20), (1, 17), (2, 10), (3, 9)]
valid [(0, 4), (1, 3), (2, 5), (3, 3)]
test [(0, 10), (1, 1), (2, 2), (3, 2)]


In [ ]:
15/85 * 